In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        #print(f"d_model : {d_model}")
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print((x + self.pe[:, :x.size(1)]).shape)
        return x + self.pe[:, :x.size(1)]

class MultiHeadCustomConvLayer(nn.Module):
    def __init__(self, input_dim, max_tokens, num_heads):
        super(MultiHeadCustomConvLayer, self).__init__()
        self.num_heads = num_heads
        self.d_k = input_dim // num_heads
        self.max_tokens = max_tokens
        
        # Define a Conv1d layer for each head
        self.convs = nn.ModuleList([nn.Conv1d(in_channels=self.d_k, 
                                              out_channels=self.d_k, 
                                              kernel_size=3, 
                                              padding=1) 
                                    for _ in range(num_heads)])
        
    def forward(self, x, curr_tokens):
        batch_size, seq_len, _ = x.size()
        
        # Split input tensor into multiple heads
        x = x.view(batch_size, self.num_heads, self.d_k, seq_len)
        x = x.transpose(2, 3)  # Change to (batch_size, num_heads, seq_len, d_k) for Conv1d
        
        outputs = []
        for i, conv in enumerate(self.convs):
            head_output = conv(x[:, i])
            outputs.append(head_output)
        
        # Combine outputs from all heads
        output = torch.cat(outputs, dim=1)  # Concatenate along the d_k dimension
        
        # Transpose and reshape back to original dimensions (with heads combined)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        
        return output

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, max_seq_length):
        super(EncoderLayer, self).__init__()
        self.custom = MultiHeadCustomConvLayer(d_model, max_seq_length, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, length):
        #print(f"X: {x}, mask: {mask}")
        attn_output = self.custom(x, length)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [4]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_emb = self.encoder_embedding(src)
        tgt_emb = self.decoder_embedding(tgt)
        src_lengths = torch.sum(src_emb != 0, dim=1)
        tgt_lengths = torch.sum(tgt_emb != 0, dim=1)

        #print("Source lengths:", src_lengths, "Max Length:", src_emb.shape)
        #print("Target lengths:", tgt_lengths)
        src_embedded = self.dropout(self.positional_encoding(src_emb))
        tgt_embedded = self.dropout(self.positional_encoding(tgt_emb))
        
        enc_output = src_embedded
        #print(src_embedded.shape)
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, 100)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(10):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")
print(transformer)

Epoch: 1, Loss: 8.695615768432617
Epoch: 2, Loss: 8.571104049682617
Epoch: 3, Loss: 8.485912322998047
Epoch: 4, Loss: 8.417162895202637
Epoch: 5, Loss: 8.358488082885742
Epoch: 6, Loss: 8.285599708557129
Epoch: 7, Loss: 8.209075927734375
Epoch: 8, Loss: 8.126957893371582
Epoch: 9, Loss: 8.044425010681152
Epoch: 10, Loss: 7.963993549346924
Transformer(
  (encoder_embedding): Embedding(5000, 512)
  (decoder_embedding): Embedding(5000, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (custom): CustomConvLayer()
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (dec

In [7]:
from transformers import BertModel, BertTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", cache_dir="../../../data2/jeongseokoh/hub/tokenizer/")
model = BertModel.from_pretrained("google-bert/bert-base-cased", cache_dir="../../../data2/jeongseokoh/hub/model/")


AttributeError: 'BertModel' object has no attribute 'BertEncoder'

In [104]:
text = "Hi my name is luke","h"
encoded = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=256)
input_ids = encoded["input_ids"]

input_ids.shape

torch.Size([2, 8])

In [105]:
emb = model.embeddings.word_embeddings(input_ids)
print(emb.shape)
class Weight(nn.Module):
    def __init__(self, d_model, tokens, curr_length):
        super(Weight, self).__init__()
        self.d_model = d_model
        self.tokens = tokens
        self.curr_length = curr_length
        self.out_dim = 3*self.tokens-1
        self.weight = nn.Parameter(torch.rand(self.tokens, self.out_dim))  # 초기화
        self.bias = nn.Parameter(torch.Tensor(self.tokens, self.out_dim))

        self._reset_parameters()
    def forward(self, x, curr_length):
        batch_size, _, _ = x.size()
weight_tensor = nn.Parameter(torch.rand(8, 15))
attn = model.encoder.layer[0].attention(emb)
print(attn[0].shape)

torch.Size([2, 8, 768])
torch.Size([2, 8, 768])


In [106]:
import torch

# 예제 데이터 생성: [batch, tokens, features] = [2, 8, 768]
example_input = torch.rand(2, 8, 768)  # 무작위 데이터

# 1단계: Transpose -> [batch, features, tokens]
transposed_input = example_input.transpose(1, 2)  # 또는 example_input.permute(0, 2, 1)

# 2단계: 가중치 텐서 생성 -> [tokens, 2*tokens-1] = [8, 15]
# 여기서는 예제로, 실제로 이러한 연산을 수행하기 위해서는 추가적인 커스텀 연산이 필요할 수 있습니다.
# 가중치 텐서를 무작위로 초기화합니다. 실제 사용 시에는 이 가중치를 학습할 수 있도록 설정해야 합니다.
weight_tensor = torch.rand(8, 15)  # 무작위 가중치 텐서

# 3단계: 가중치 텐서와의 곱 -> [batch, features, 2*tokens-1]
# 이 단계는 실제 행렬 곱셈 연산과 다를 수 있으며, 적절한 차원 변환을 위한 추가 작업이 필요할 수 있습니다.
# 예제 코드는 이론적 설명을 위한 것이며, 실제 구현에는 커스텀 로직이 필요할 수 있습니다.
output = torch.matmul(transposed_input, weight_tensor)

# 결과 확인
output_shape = output.shape
output_shape

torch.Size([2, 768, 15])

In [107]:
import torch

# 예제 데이터 생성: [batch, tokens, features] = [2, 256, 768]
example_input = torch.rand(2, 256, 768)  # 무작위 데이터

# 1단계: Transpose -> [batch, features, tokens]
transposed_input = example_input.transpose(1, 2)  # 또는 example_input.permute(0, 2, 1)

# 2단계: 가중치 행렬 생성 -> [tokens, 2*tokens-1] = [256, 511]
weight_matrix = torch.rand(256, 511)  # 무작위 가중치 행렬

# 3단계: 가중치 행렬과의 곱 -> [batch, features, 2*tokens-1]
output = torch.matmul(transposed_input, weight_matrix)

# 결과 확인
output_shape = output.shape
output_shape

torch.Size([2, 768, 511])

In [114]:
# 계산량 비교를 위한 파라미터 설정
seq_len = 8  # 시퀀스 길이
d_model = 768  # 임베딩 차원

# Attention 계산량
attention_calc = 3 * seq_len * d_model**2 + seq_len**2 * d_model

# 사용자 정의 가중치 연산의 계산량
custom_weight_calc = seq_len * (100 * seq_len - 1) * d_model

attention_calc, custom_weight_calc

(14204928, 4909056)